# Split Dataset Using Topic Chooser
#### 29 Nov 2020

In [1]:
# Set Evaulation Parameters
MODEL_NAME = "distilbert-base-uncased"
MODEL_FILE ="saved_model_epoch5_20201126_1904_w1_4.tar"
DATASET = "train_dataset_25Nov_source_distilbert.pickle"

PF_DATASET = "val_pf_sorted_dataset_distilbert_titles.pickle"
GC_DATASET = "val_gc_sorted_dataset_distilbert_titles.pickle"

In [2]:
import copy
import itertools
import pickle
import sys

import json
import pandas as pd
import pickle
import tqdm
import torch
import torch.utils.data
import transformers as hft

sys.path.insert(0, "/home/jupyter")
import util.log
import util.data

In [3]:
# Load dataset that will be split
with open(DATASET, "rb") as dfile:
    dataset = pickle.load(dfile)

In [4]:
# Load saved model
checkpoint = torch.load(MODEL_FILE)
model = (hft.DistilBertForSequenceClassification
         .from_pretrained(MODEL_NAME))
model.load_state_dict(checkpoint["model"])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

<All keys matched successfully>

In [5]:
# Move model to GPU if available
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
model.to(device)
model.eval();
device

device(type='cuda')

In [6]:
is_political = []

with torch.no_grad():
    for idx, article in tqdm.tqdm(enumerate(dataset)):
        input_ids = article["input_ids"].to(device)
        attent_mask = article["attention_mask"].to(device)
        output = model(input_ids.unsqueeze(0),
                       attention_mask=attent_mask.unsqueeze(0),
                       output_hidden_states=False,
                       output_attentions=False)
        logit = output[0].squeeze(0).detach().cpu().numpy()
        is_political.append(logit[1] >= logit[0])

0it [00:00, ?it/s]/home/jupyter/util/data.py:458: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
10986it [08:15, 22.16it/s]


In [7]:
polit_data = util.data.subset_FNDataset(dataset, is_political)

In [8]:
gossip_data = util.data.subset_FNDataset(dataset, is_political, inverse=True)

In [9]:
assert len(polit_data) + len(gossip_data) == len(dataset)

In [10]:
len(polit_data)

406

In [11]:
len(gossip_data)

10580

In [15]:
with open("political_files_train.pickle", "wb") as pfile:
    pickle.dump(polit_data.file_names, pfile)

In [16]:
with open("gossip_files_train.pickle", "wb") as pfile:
    pickle.dump(gossip_data.file_names, pfile)